<a href="https://colab.research.google.com/github/jain-abhinav02/Text-Generation/blob/master/Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# import the modules
import io
import numpy as np
import random
from keras.models import Sequential
from keras.layers import Dense,LSTM
from keras.callbacks import LambdaCallback

In [0]:
# load the text into a string text
# here, cp1252 encoding scheme is used
path = "drive/My Drive/Datasets/alice in wonderland.txt"
with io.open(path, encoding='cp1252') as f:
    text = f.read().lower()

In [5]:
# the size of the input text
print(len(text))

144435


In [6]:
# let us fix input sentence length to maxlen
# create a number of sample sentence with the next characters as output
maxlen = 40
step =3
sentences= []
out_chars=[]
for i in range(0,len(text)-maxlen,step):
  sentences.append(text[i:i+maxlen])
  out_chars.append(text[i+maxlen])
print("No of samples= ",len(sentences))

No of samples=  48132


In [0]:
# a glimpse of the sampling
for i in range(20):
  print(sentences[i],' ',out_chars[i])

In [10]:
# the set of all characters used in text
list_chars=list(set(text))
print(list_chars)

['(', 'r', ']', 'p', '!', 'z', 't', 'i', 'y', 's', 'l', 'd', ',', '*', ';', 'q', '“', 'n', 'o', 'h', ' ', 'c', 'w', '-', '”', 'm', '’', 'u', '‘', 'x', '.', 'a', '_', 'e', 'g', 'b', '[', 'v', ')', '3', 'j', '\n', 'k', 'f', '0', ':', '?']


In [11]:
# count of dictionary to be created
print(len(list_chars))

47


In [0]:
# create two dictionaries mapping characters to indices and vice versa
char2ind=dict()
ind2char=dict()
for i in range(len(list_chars)):
  ch=list_chars[i]
  char2ind[ch]=i
  ind2char[i]=ch

In [13]:
char2ind.items()

dict_items([('(', 0), ('r', 1), (']', 2), ('p', 3), ('!', 4), ('z', 5), ('t', 6), ('i', 7), ('y', 8), ('s', 9), ('l', 10), ('d', 11), (',', 12), ('*', 13), (';', 14), ('q', 15), ('“', 16), ('n', 17), ('o', 18), ('h', 19), (' ', 20), ('c', 21), ('w', 22), ('-', 23), ('”', 24), ('m', 25), ('’', 26), ('u', 27), ('‘', 28), ('x', 29), ('.', 30), ('a', 31), ('_', 32), ('e', 33), ('g', 34), ('b', 35), ('[', 36), ('v', 37), (')', 38), ('3', 39), ('j', 40), ('\n', 41), ('k', 42), ('f', 43), ('0', 44), (':', 45), ('?', 46)])

In [15]:
print(char2ind['a'])

31


In [16]:
print(ind2char.items())

dict_items([(0, '('), (1, 'r'), (2, ']'), (3, 'p'), (4, '!'), (5, 'z'), (6, 't'), (7, 'i'), (8, 'y'), (9, 's'), (10, 'l'), (11, 'd'), (12, ','), (13, '*'), (14, ';'), (15, 'q'), (16, '“'), (17, 'n'), (18, 'o'), (19, 'h'), (20, ' '), (21, 'c'), (22, 'w'), (23, '-'), (24, '”'), (25, 'm'), (26, '’'), (27, 'u'), (28, '‘'), (29, 'x'), (30, '.'), (31, 'a'), (32, '_'), (33, 'e'), (34, 'g'), (35, 'b'), (36, '['), (37, 'v'), (38, ')'), (39, '3'), (40, 'j'), (41, '\n'), (42, 'k'), (43, 'f'), (44, '0'), (45, ':'), (46, '?')])


In [18]:
print(ind2char[31])

a


In [0]:
# vectorise the input samples using numpy arrays
x = np.zeros((len(sentences),maxlen,len(list_chars)),dtype=np.bool)
y = np.zeros((len(sentences),len(list_chars)),dtype=np.bool)

In [0]:
# using dictionary to convert sample sentences into numpy arrays
for i,sentence in enumerate(sentences):
  for j,char in enumerate(sentence):
    x[i,j,char2ind[char]]=True
  y[i,char2ind[out_chars[i]]]=True

In [22]:
print(x[0])

[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]


In [23]:
print(x[0,0])

[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False  True False False False False
 False False False False False False False False False False False]


In [24]:
print(y[0])

[False False False False False False False False False False False False
 False False False False False False False False False  True False False
 False False False False False False False False False False False False
 False False False False False False False False False False False]


In [25]:
print(type(x))

<class 'numpy.ndarray'>


In [29]:
# build the model
model=Sequential()
model.add(LSTM(128,input_shape=(maxlen,len(list_chars))))
model.add(Dense(len(list_chars),activation='softmax'))

W0625 04:36:40.330406 139944339470208 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0625 04:36:40.345288 139944339470208 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [0]:
# generate text
def gen_text(epochs,_):
  seed=random.randint(0,len(text)-maxlen)
  base=text[seed:seed+maxlen]
  print("Base sentence ",base,end='....')
  for i in range(200):
    test_x=np.zeros((1,maxlen,len(list_chars)))
    for j,char in enumerate(base):
      test_x[0,j,char2ind[char]]=1.0
    res=model.predict(test_x)[0]
    res=np.asarray(res).astype('float64')
    res/=np.sum(res)
    res=np.random.multinomial(1,res,1)
    test_y=ind2char[np.argmax(res)]
    base=base[1:]+test_y
    print(test_y,end='')
  print()

In [0]:
cb_print=LambdaCallback(on_epoch_end=gen_text)

In [0]:
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [67]:
model.fit(x,y,epochs=2,batch_size=64,callbacks=[cb_print])

Epoch 1/2
48132/48132 [==============================] - 51s 1ms/step - loss: 0.6688
Base sentence  iver all over with fright.
‘oh, i beg yo....urseld, she went on? i should lizes had no secropfulers, but the fourt you?’ the pattire.

‘are they moled all make your pacery! sleessy quite nold all lay
a toff
intabo uping the jory lasters.

‘as t
Epoch 2/2
48132/48132 [==============================] - 51s 1ms/step - loss: 0.6538
Base sentence  said nothing.

‘perhaps it doesn’t under....sent and
all, and a long
to he was quite, and exacklonwardly teal his a
a long to herself. ‘and on youre leves this.’

‘--and then, you geg,’ said the cat, ‘low as the look hisseed a moment. ‘it too b
